<a href="https://colab.research.google.com/github/utsav-195/sentiment-analysis-using-rnn-in-pytorch/blob/main/Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!tar -xvf "/content/drive/My Drive/D4NLP/Ass 1/rt-polaritydata.tar.gz" -C "/content/drive/My Drive/D4NLP/Ass 1/"

tar: option requires an argument -- 'f'
Try 'tar --help' or 'tar --usage' for more information.
rt-polaritydata.README.1.0.txt
rt-polaritydata/rt-polarity.neg
rt-polaritydata/rt-polarity.pos


In [1]:
!pip install pyprind

In [2]:
import numpy as np
import pandas as pd
import torch
from torchtext import data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torch.autograd import Variable
from tqdm import tqdm
import nltk

import random
from sklearn.metrics import classification_report

import pyprind
%matplotlib inline  

#Pre-Processing

In [ ]:
data_pos = []
with open("/content/drive/My Drive/D4NLP/Ass 1/rt-polaritydata/rt-polarity.pos",encoding='latin-1') as f:
  for line in f:
    line  = line[:-2]
    line = line.replace("'t","t")
    line = line.replace("'"," '")
    line = line.replace("/"," / ")
    line = line.replace("-"," - ")
    line = line.replace("["," [ ")
    line = line.replace("]"," ] ")
    data_pos.append(line)

In [ ]:
# creating train, dev, test split positive reviews
data_pos_train = data_pos[:int(len(data_pos)*0.6)]
data_pos_dev = data_pos[int(len(data_pos)*0.6):int(len(data_pos)*0.8)]
data_pos_test = data_pos[int(len(data_pos)*0.8):]

data_pos_train = np.asarray(data_pos_train)
data_pos_dev = np.asarray(data_pos_dev)
data_pos_test = np.asarray(data_pos_test)

y_pos_train = np.asarray([1]*len(data_pos_train))
y_pos_dev = np.asarray([1]*len(data_pos_dev))
y_pos_test = np.asarray([1]*len(data_pos_test))

print(data_pos_train.shape)
print(data_pos_dev.shape)
print(data_pos_test.shape)

(3198,)
(1066,)
(1067,)


In [ ]:
# reading the negative reviews
data_neg = []
with open("/content/drive/My Drive/D4NLP/Ass 1/rt-polaritydata/rt-polarity.neg",encoding='latin-1') as f:
    for line in f:
        line  = line[:-2]
        line = line.replace("'t","t")
        line = line.replace("'"," '")
        line = line.replace("/"," / ")
        line = line.replace("-"," - ")
        line = line.replace("["," [ ")
        line = line.replace("]"," ] ")
        data_neg.append(line)

In [ ]:
# creating train, dev, test split for negative reviews
data_neg_train = data_neg[:int(len(data_neg)*0.6)]
data_neg_dev = data_neg[int(len(data_neg)*0.6):int(len(data_neg)*0.8)]
data_neg_test = data_neg[int(len(data_neg)*0.8):]

data_neg_train = np.asarray(data_neg_train)
data_neg_dev = np.asarray(data_neg_dev)
data_neg_test = np.asarray(data_neg_test)

y_neg_train = np.asarray([0]*len(data_neg_train))
y_neg_dev = np.asarray([0]*len(data_neg_dev))
y_neg_test = np.asarray([0]*len(data_neg_test))

print(data_neg_train.shape)
print(data_neg_dev.shape)
print(data_neg_test.shape)

(3198,)
(1066,)
(1067,)


In [ ]:
x_train = np.concatenate((data_pos_train, data_neg_train), axis=0)
x_dev = np.concatenate((data_pos_dev, data_neg_dev), axis=0)
x_test = np.concatenate((data_pos_test, data_neg_test), axis=0)

y_train = np.concatenate((y_pos_train, y_neg_train), axis=0)
y_dev = np.concatenate((y_pos_dev, y_neg_dev), axis=0)
y_test = np.concatenate((y_pos_test, y_neg_test), axis=0)

print(x_train.shape)
print(y_train.shape)


(6396,)
(6396,)


In [ ]:
def seq_length(data,label,seq_len):
  data_seq_len = []
  for i in range(len(data)):
    data_seq_len.append(" ".join(data[i].split()[:seq_len]))
  df = pd.DataFrame()
  df['reviews'] = data_seq_len
  df['senti'] = label

  df = df.sample(frac = 1).reset_index(drop=True)

  return df

In [ ]:
seq_len = 35
df_train = seq_length(x_train,y_train,seq_len)
df_dev = seq_length(x_dev,y_dev,seq_len)
df_test = seq_length(x_test,y_test,seq_len)

In [ ]:
!mkdir "/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/"

In [ ]:
df_train.to_csv("/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/train.csv", index=False)
df_test.to_csv("/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/test.csv", index=False)
df_dev.to_csv("/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/dev.csv", index=False)

# Referenced

https://www.kaggle.com/kuldeep7688/simple-rnn-using-glove-embeddings-in-pytorch

https://github.com/bentrevett/pytorch-sentiment-analysis

In [3]:
# sample tokenizer which you can use
def tokenizer(text):
    return [tok for tok in text.split()]

In [4]:
# tokenizer = "spacy" uses spacy's tokenizer
TEXT = data.Field(sequential=True, tokenize=tokenizer)
LABEL = data.LabelField(dtype=torch.long, sequential=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# loading train, test and validation data 
train_data, dev_data, test_data = data.TabularDataset.splits(
    path="/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/", train="train.csv", 
    validation="dev.csv", test="test.csv",format="csv", skip_header=True, 
    fields=[('reviews', TEXT), ('senti', LABEL)]
)

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of valid examples: {len(dev_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 6396
Number of valid examples: 2132
Number of testing examples: 2134


In [8]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors("/content/drive/My Drive/D4NLP/Ass 1/glove.6B.300d.txt"), 
                 max_size=40000, min_freq=1)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 14410
Unique tokens in LABEL vocabulary: 2


In [10]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, dev_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, dev_data, test_data), sort_key=lambda x: len(x.reviews),
    batch_size=BATCH_SIZE,
    device=device)

In [11]:
for i in train_iterator:
  print(i)
  break


[torchtext.data.batch.Batch of size 32]
	[.reviews]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
	[.senti]:[torch.cuda.LongTensor of size 32 (GPU 0)]


## RNN

In [12]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        # print(x.shape)
        # print(embedded.shape)
        #pack sequence
        # packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)

        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        hidden = self.dropout(hidden)
        
        out = self.fc(hidden)
        return out

In [13]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 2
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model_rnn = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            DROPOUT)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [16]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model_rnn.embedding.weight.data = pretrained_embeddings.cuda()

class_weights = torch.tensor([1.0, 1.0]).cuda()

optimizer = optim.Adam(model_rnn.parameters(), lr=0.00003)

criterion = nn.CrossEntropyLoss(weight=class_weights)

model_rnn = model_rnn.to(device)
criterion = criterion.to(device)

torch.Size([14410, 300])


In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    preds, ind= torch.max(F.softmax(preds, dim=-1), 1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))

    return acc

In [18]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    bar = pyprind.ProgBar(len(iterator), bar_char='█')
    for batch in iterator:
        
        optimizer.zero_grad()
        # batch, batch_lengths = batch.reviews

        predictions = model(batch.reviews).squeeze(0)
#         print(predictions.shape, batch.Label.shape, model(batch.Text).shape)
        # print("idar")
        # print(predictions.shape)
        # print(batch.senti.reshape((len(batch.senti),1)).shape)

        loss = criterion(predictions, batch.senti)

        acc = binary_accuracy(predictions, batch.senti)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        bar.update()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        bar = pyprind.ProgBar(len(iterator), bar_char='█')
        for batch in iterator:

            predictions = model(batch.reviews).squeeze(0)
            
            loss = criterion(predictions, batch.senti)
            
            acc = binary_accuracy(predictions, batch.senti)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            bar.update()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
N_EPOCHS = 100
train_rnn_loss = []
train_rnn_acc = []
dev_rnn_loss = []
dev_rnn_acc = []


for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model_rnn, train_iterator, optimizer, criterion)
    dev_loss, dev_acc = evaluate(model_rnn, dev_iterator, criterion)
    
    train_rnn_loss.append(train_loss)
    train_rnn_acc.append(train_acc)

    dev_rnn_loss.append(dev_loss)
    dev_rnn_acc.append(dev_acc)


    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {dev_loss:.3f} | Val. Acc: {dev_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 01 | Train Loss: 0.695 | Train Acc: 49.39% | Val. Loss: 0.697 | Val. Acc: 50.32% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 02 | Train Loss: 0.693 | Train Acc: 50.04% | Val. Loss: 0.695 | Val. Acc: 50.46% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 99 | Train Loss: 0.230 | Train Acc: 91.41% | Val. Loss: 0.661 | Val. Acc: 72.71% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 100 | Train Loss: 0.215 | Train Acc: 91.89% | Val. Loss: 0.663 | Val. Acc: 71.56% |



Total time elapsed: 00:00:00


## LSTM

In [21]:
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        # print("here1")
        #embedded = [sent len, batch size, emb dim]
        # print(x.shape)
        # print(embedded.shape)
        output, (hidden,cell) = self.rnn(embedded)
        # print("here2")
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        # print(hidden)
        # print(hidden[0].shape)
        hidden = self.dropout(hidden)
        # print("here3")
        out = self.fc(hidden)
        # print("here4")
        return out

In [22]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 2
DROPOUT = 0.5

model_lstm = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [23]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model_lstm.embedding.weight.data = pretrained_embeddings

class_weights = torch.tensor([1.0, 1.0])

optimizer = optim.Adam(model_lstm.parameters(), lr=0.00003)

criterion = nn.CrossEntropyLoss(weight=class_weights)

model_lstm = model_lstm.to(device)
criterion = criterion.to(device)

torch.Size([14410, 300])


In [24]:
N_EPOCHS = 100
train_lstm_loss = []
train_lstm_acc = []
dev_lstm_loss = []
dev_lstm_acc = []

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model_lstm, train_iterator, optimizer, criterion)
    dev_loss, dev_acc = evaluate(model_lstm, dev_iterator, criterion)
    
    train_lstm_loss.append(train_loss)
    train_lstm_acc.append(train_acc)

    dev_lstm_loss.append(dev_loss)
    dev_lstm_acc.append(dev_acc)

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {dev_loss:.3f} | Val. Acc: {dev_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 01 | Train Loss: 0.694 | Train Acc: 49.69% | Val. Loss: 0.693 | Val. Acc: 51.10% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 02 | Train Loss: 0.693 | Train Acc: 50.17% | Val. Loss: 0.692 | Val. Acc: 54.81% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 99 | Train Loss: 0.144 | Train Acc: 94.23% | Val. Loss: 0.595 | Val. Acc: 78.70% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 100 | Train Loss: 0.152 | Train Acc: 94.00% | Val. Loss: 0.589 | Val. Acc: 78.42% |



Total time elapsed: 00:00:00


##GRU

In [25]:
class GRU(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        # print("here1")
        #embedded = [sent len, batch size, emb dim]
        # print(x.shape)
        # print(embedded.shape)
        output, hidden = self.rnn(embedded)
        # print("here2")
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        # print(hidden)
        # print(hidden[0].shape)
        hidden = self.dropout(hidden)
        # print("here3")
        out = self.fc(hidden)
        # print("here4")
        return out

In [26]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 2
DROPOUT = 0.4

model_gru = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [27]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model_gru.embedding.weight.data = pretrained_embeddings

class_weights = torch.tensor([1.0, 1.0])

optimizer = optim.Adam(model_gru.parameters(), lr=0.00003)

criterion = nn.CrossEntropyLoss(weight=class_weights)

model_gru = model_gru.to(device)
criterion = criterion.to(device)

torch.Size([14410, 300])


In [28]:
N_EPOCHS = 100
train_gru_loss = []
train_gru_acc = []
dev_gru_loss = []
dev_gru_acc = []

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model_gru, train_iterator, optimizer, criterion)
    dev_loss, dev_acc = evaluate(model_gru, dev_iterator, criterion)
    
    train_gru_loss.append(train_loss)
    train_gru_acc.append(train_acc)

    dev_gru_loss.append(dev_loss)
    dev_gru_acc.append(dev_acc)

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {dev_loss:.3f} | Val. Acc: {dev_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 01 | Train Loss: 0.694 | Train Acc: 49.18% | Val. Loss: 0.692 | Val. Acc: 51.61% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 02 | Train Loss: 0.693 | Train Acc: 51.13% | Val. Loss: 0.690 | Val. Acc: 54.81% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 99 | Train Loss: 0.098 | Train Acc: 96.01% | Val. Loss: 0.742 | Val. Acc: 76.23% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 100 | Train Loss: 0.099 | Train Acc: 96.28% | Val. Loss: 0.634 | Val. Acc: 77.56% |



Total time elapsed: 00:00:00


##Evaluation

In [38]:
def predict_sentiment(model,sentence):
    tokenized = [tok for tok in sentence.split()]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    
    tensor = tensor.unsqueeze(1)
#     print(tensor.shape)
    prediction = model(tensor)
#     print(prediction)
    preds, ind= torch.max(F.softmax(prediction.squeeze(0), dim=-1), 1)
#     print(preds)
    return preds, ind

In [45]:
# calculating classification report
test_data = pd.read_csv("/content/drive/My Drive/D4NLP/Ass 1/torchtext_data/test.csv")

In [46]:
pre = [predict_sentiment(model_rnn,k)[1].item() for k in test_data.reviews]
print(classification_report(test_data.senti, pre))

              precision    recall  f1-score   support

           0       0.70      0.74      0.72      1067
           1       0.72      0.69      0.70      1067

    accuracy                           0.71      2134
   macro avg       0.71      0.71      0.71      2134
weighted avg       0.71      0.71      0.71      2134



In [47]:
pre = [predict_sentiment(model_lstm,k)[1].item() for k in test_data.reviews]
print(classification_report(test_data.senti, pre))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78      1067
           1       0.78      0.77      0.78      1067

    accuracy                           0.78      2134
   macro avg       0.78      0.78      0.78      2134
weighted avg       0.78      0.78      0.78      2134



In [48]:
pre = [predict_sentiment(model_gru,k)[1].item() for k in test_data.reviews]
print(classification_report(test_data.senti, pre))

              precision    recall  f1-score   support

           0       0.79      0.74      0.76      1067
           1       0.76      0.80      0.78      1067

    accuracy                           0.77      2134
   macro avg       0.77      0.77      0.77      2134
weighted avg       0.77      0.77      0.77      2134



# Custom

In [ ]:
!wget -O "/content/drive/My Drive/D4NLP/Ass 1/glove.zip" http://nlp.stanford.edu/data/glove.6B.zip

--2020-10-19 22:05:01--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-10-19 22:05:01--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-10-19 22:05:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/content/drive/My Driv

In [ ]:
!unzip "/content/drive/My Drive/D4NLP/Ass 1/glove.zip" -d "/content/drive/My Drive/D4NLP/Ass 1"

Archive:  /content/drive/My Drive/D4NLP/Ass 1/glove.zip
  inflating: /content/drive/My Drive/D4NLP/Ass 1/glove.6B.50d.txt  
  inflating: /content/drive/My Drive/D4NLP/Ass 1/glove.6B.100d.txt  
  inflating: /content/drive/My Drive/D4NLP/Ass 1/glove.6B.200d.txt   (warning) cannot set modif./access times
          No such file or directory
  inflating: /content/drive/My Drive/D4NLP/Ass 1/glove.6B.300d.txt  


In [ ]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    model = {}
    for line in f:
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            print(word)
    print("Done.",len(model)," words loaded!")
    return model

In [ ]:
glove_embeddings = loadGloveModel("/content/drive/My Drive/D4NLP/Ass 1/glove.6B.300d.txt")

Loading Glove Model
Done. 400000  words loaded!


In [ ]:
def create_embedding(data,sequence_length):
  embedding_matrix = np.zeros((len(data),sequence_length,300))
  for d in range(len(data)):
    embed = []
    tokens = data[d].split()
    embed_length = min(sequence_length,len(tokens))
    for token in range(embed_length):
      try:
        embedding_matrix[d][token] = glove_embeddings[tokens[token]]
      except:
        print("Word not in glove",tokens[token])
  return torch.from_numpy(embedding_matrix).float()

In [ ]:
sequence_length = 35

embedding_x_train = create_embedding(x_train,sequence_length)
embedding_x_dev = create_embedding(x_dev,sequence_length)
embedding_x_test = create_embedding(x_test,sequence_length)

Word not in glove biopi
Word not in glove wisegirls
Word not in glove 'who
Word not in glove enrapturing
Word not in glove 'compleja
Word not in glove intelectualmente
Word not in glove retadora
Word not in glove orquídeas
Word not in glove originalidad
Word not in glove 
Word not in glove 
Word not in glove 'performance
Word not in glove movi
Word not in glove sarcas
Word not in glove suspenser
Word not in glove obviation
Word not in glove gorefests
Word not in glove waydowntown
Word not in glove 
Word not in glove makmalbaf
Word not in glove 'urban
Word not in glove exhilarate
Word not in glove nuttgens
Word not in glove 'a
Word not in glove 
Word not in glove petin
Word not in glove provocatuers
Word not in glove jirí
Word not in glove hubac
Word not in glove shapelessly
Word not in glove theocrac
Word not in glove 
Word not in glove agai
Word not in glove 'bartleby
Word not in glove werent
Word not in glove addessi
Word not in glove seldahl
Word not in glove wollter
Word not i

In [ ]:
print(embedding_x_train.shape)
print(embedding_x_dev.shape)
print(embedding_x_test.shape)

torch.Size([6396, 35, 300])
torch.Size([2132, 35, 300])
torch.Size([2134, 35, 300])


In [ ]:
y_train = torch.from_numpy(y_train)
y_dev = torch.from_numpy(y_dev)
y_test = torch.from_numpy(y_test)

In [ ]:
import sklearn

embedding_x_train, y_train = sklearn.utils.shuffle(embedding_x_train, y_train)
embedding_x_dev, y_dev = sklearn.utils.shuffle(embedding_x_dev, y_dev)

In [ ]:
embedding_x_train, y_train, embedding_x_dev, y_dev = embedding_x_train.cuda(), y_train.cuda(), embedding_x_dev.cuda(), y_dev.cuda()

In [ ]:
class RNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, dropout):
        super(RNN,self).__init__()
        self.hidden_dim = hidden_dim
        # self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, dropout=dropout, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_dim, output_dim)
        # self.fc = nn.Sequential(nn.Linear(hidden_dim, output_dim),nn.Sigmoid())

        self.dropout = nn.Dropout(dropout)
    
    def __init__hidden(self, batch_size):
        hidden = torch.zeros(1,batch_size, self.hidden_dim).cuda()
        return Variable(hidden)

    def forward(self, x):

        #x = [sent len, batch size]
        
        # embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        # print("x",x.shape)
        hidden = self.__init__hidden(len(x[0]))
        output, hidden = self.rnn(x,hidden)
        # print("out:",output.shape)
        # print("hidden:",hidden.shape)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        hidden = self.dropout(hidden)
        # print("hidden2:",hidden.shape)
        out = self.fc(hidden)
        # print("out2:",out.shape)
        return out

In [ ]:
# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 2
DROPOUT = 0.4

model = RNN(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT).cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=0.00003,weight_decay=0.001)

In [ ]:
class_weights = torch.tensor([1.0, 1.0]).cuda()

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights).cuda()

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    preds, ind= torch.max(F.softmax(preds, dim=-1), 1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:
def evaluate(model, x, y, batch_size, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    
    with torch.no_grad():
        bar = pyprind.ProgBar(int(len(x)/batch_size), bar_char='█')
        # for batch in iterator:
        for batch_index in tqdm(range(int(len(x)/batch_size))):
            batch, target = x[batch_index*batch_size : batch_index*batch_size + batch_size], y[batch_index*batch_size : batch_index*batch_size + batch_size]
            
            batch = torch.reshape(batch, (sequence_length,batch_size, 300))

            predictions = model(batch)
            
            loss = criterion(predictions[0], target)
            
            acc = binary_accuracy(predictions[0], target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            bar.update()
    return epoch_loss / (int(len(x)/batch_size)), epoch_acc / (int(len(x)/batch_size))

In [ ]:
def train(model, x, y, batch_size, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    bar = pyprind.ProgBar(int(len(x)/batch_size), bar_char='█')
    for batch_index in range(int(len(x)/batch_size)):
        optimizer.zero_grad()
        # print(np.asarray([x[batch_index : batch_index + batch_size]]).shape)
        batch, target = x[batch_index*batch_size : batch_index*batch_size + batch_size], y[batch_index*batch_size : batch_index*batch_size + batch_size]
            
        batch = torch.reshape(batch, (sequence_length,batch_size, 300))

        batch, target = Variable(batch), Variable(target)

        predictions = model(batch)
        # print(len(target))
        # print("naya")
        # print(predictions[0].shape)
        # print(target.shape)
#         print(predictions.shape, batch.Label.shape, model(batch.Text).shape)
        # print(predictions)
        # print(target)
        loss = criterion(predictions[0], target)
        
        # print(loss.shape)
        acc = binary_accuracy(predictions[0], target)
        # print(acc)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        bar.update()
    return epoch_loss / (int(len(x)/batch_size)), epoch_acc / (int(len(x)/batch_size))

In [ ]:
N_EPOCHS = 200
BATCH_SIZE = 64

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, embedding_x_train, y_train, BATCH_SIZE, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, embedding_x_dev, y_dev, BATCH_SIZE, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 301.73it/s]

| Epoch: 01 | Train Loss: 0.696 | Train Acc: 50.41% | Val. Loss: 0.695 | Val. Acc: 49.38% |



0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 231.79it/s]

| Epoch: 02 | Train Loss: 0.693 | Train Acc: 52.08% | Val. Loss: 0.696 | Val. Acc: 49.72% |



0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 264.99it/s]

...

| Epoch: 198 | Train Loss: 0.086 | Train Acc: 97.82% | Val. Loss: 4.084 | Val. Acc: 48.82% |



0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 257.86it/s]

| Epoch: 199 | Train Loss: 0.079 | Train Acc: 98.18% | Val. Loss: 4.102 | Val. Acc: 49.15% |



0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 298.07it/s]

| Epoch: 200 | Train Loss: 0.076 | Train Acc: 98.26% | Val. Loss: 4.158 | Val. Acc: 48.86% |


In [ ]:
test_loss, test_acc = evaluate(model, embedding_x_test.cuda(), y_test.cuda(), BATCH_SIZE, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
100%|██████████| 33/33 [00:00<00:00, 286.15it/s]

| Test Loss: 2.920 | Test Acc: 49.05% |


In [ ]:
x_test[:2]

array(["it 's clear that mehta simply wanted to update her beloved genre for the thousands of indians who fancy themselves too sophisticated for the cheese - laced spectacles that pack  'em in on the subcontinent .",
       'compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .'],
      dtype='<U266')

In [ ]:
test_ut = ["the movie was horrible .",
           "awesome movie . greatest ever built ."]

In [ ]:
embed_ut = create_embedding(test_ut,sequence_length)

In [ ]:
out = model(embed_ut)